# ATM_은행
- 다른 프로세스 종료될 때까지 기다리는 시스템
- ATM 대기열 : `Store`

제한된 수의 `ATM 기계`와 랜덤하게 도착하는 `고객`으로 이루어진 은행 ATM 시뮬레이션 환경을 생성합니다.

`ATM 대기열`은 `Store`를 사용하여 제한된 수의 `ATM 기계`를 모델링합니다.

고객이 도착하면 ATM 대기열에 서게 됩니다. 만약 이미 줄에 서있는 사람의 수가 `queue.capacity` 이하라면 도착한 고객도 줄을 서겠지만, 그 이상이라면 다음에 ATM을 사용할 계획으로 줄을 서지도 않고 대기열을 이탈합니다.

본인 차례가 되면 ATM 기계를 사용하며, 사용 종료 이후 은행을 떠납니다.

In [1]:
import simpy
import random

class ATM(object):
    def __init__(self, env):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
    
    def use(self, customer):
        yield self.env.timeout(random.randint(1, 5))


In [13]:
class Bank(object):
    def __init__(self, env, num_ATM, queue_len):
        self.env = env
        self.atms = [ATM(env) for _ in range(num_ATM)]
        self.queue = simpy.resources.store.Store(env, capacity=queue_len)
    
    def arrive(self, customer):
        if len(self.queue.items) < self.queue.capacity:
            print(f"({self.env.now:.1f}){customer}이 은행에 도착하였습니다.")
            self.queue.put(customer)
            with self.atms[0].machine.request() as req:
                yield req
                self.queue.get()
                print(f"({self.env.now:.1f}) {customer} ATM 사용 시작!")
                yield self.env.process(self.atms[0].use(customer))
                print(f"({self.env.now:.1f}) {customer} ATM 사용 종료!")
        else:
            print(f"({self.env.now:.1f}){customer}이 ATM에 도착했으나, 줄이 길어서 이탈!")

In [15]:
def customer_arrivals(env, bank):
    customer_num = 0
    while True:
        yield env.timeout(random.expovariate(1/3))
        customer = f"고객 {customer_num}"
        customer_num += 1
        env.process(bank.arrive(customer))

In [18]:
env = simpy.Environment()
bank = Bank(env, num_ATM=3, queue_len=5)
env.process(customer_arrivals(env, bank))
env.run(until=50)

(2.9)고객 0이 은행에 도착하였습니다.
(2.9) 고객 0 ATM 사용 시작!
(4.9) 고객 0 ATM 사용 종료!
(10.4)고객 1이 은행에 도착하였습니다.
(10.4) 고객 1 ATM 사용 시작!
(12.4) 고객 1 ATM 사용 종료!
(14.9)고객 2이 은행에 도착하였습니다.
(14.9) 고객 2 ATM 사용 시작!
(15.9)고객 3이 은행에 도착하였습니다.
(16.9)고객 4이 은행에 도착하였습니다.
(18.1)고객 5이 은행에 도착하였습니다.
(19.9) 고객 2 ATM 사용 종료!
(19.9) 고객 3 ATM 사용 시작!
(23.2)고객 6이 은행에 도착하였습니다.
(23.2)고객 7이 은행에 도착하였습니다.
(24.6)고객 8이 은행에 도착하였습니다.
(24.9) 고객 3 ATM 사용 종료!
(24.9) 고객 4 ATM 사용 시작!
(26.9) 고객 4 ATM 사용 종료!
(26.9) 고객 5 ATM 사용 시작!
(28.3)고객 9이 은행에 도착하였습니다.
(30.9) 고객 5 ATM 사용 종료!
(30.9) 고객 6 ATM 사용 시작!
(31.2)고객 10이 은행에 도착하였습니다.
(31.9) 고객 6 ATM 사용 종료!
(31.9) 고객 7 ATM 사용 시작!
(32.2)고객 11이 은행에 도착하였습니다.
(36.3)고객 12이 은행에 도착하였습니다.
(36.6)고객 13이 ATM에 도착했으나, 줄이 길어서 대기 중!
(36.9) 고객 7 ATM 사용 종료!
(36.9) 고객 8 ATM 사용 시작!
(38.9) 고객 8 ATM 사용 종료!
(38.9) 고객 9 ATM 사용 시작!
(40.1)고객 14이 은행에 도착하였습니다.
(40.7)고객 15이 은행에 도착하였습니다.
(40.9) 고객 9 ATM 사용 종료!
(40.9) 고객 10 ATM 사용 시작!
(43.2)고객 16이 은행에 도착하였습니다.
(43.8)고객 17이 ATM에 도착했으나, 줄이 길어서 대기 중!
(43.9)고객 18이 ATM에 도착했으나, 줄이 길어서 대기 중!